In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import tqdm
import re
from sklearn.multioutput import MultiOutputRegressor
import seaborn as sns
import matplotlib.pyplot as plt

from warnings import simplefilter

import matplotlib.pyplot as plt
import lightgbm as lgb

from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import os

import seaborn as sns; sns.set()
import gc

from sklearn import preprocessing

from typing import Union
from tqdm.notebook import tqdm_notebook as tqdm

import warnings
warnings.simplefilter('ignore')

In [2]:
groups = [['HOBBIES', 'CA_1'],
       ['HOUSEHOLD', 'CA_1'],
       ['FOODS', 'CA_1'],
       ['HOBBIES', 'CA_2'],
       ['HOUSEHOLD', 'CA_2'],
       ['FOODS', 'CA_2'],
       ['HOBBIES', 'CA_3'],
       ['HOUSEHOLD', 'CA_3'],
       ['FOODS', 'CA_3'],
       ['HOBBIES', 'CA_4'],
       ['HOUSEHOLD', 'CA_4'],
       ['FOODS', 'CA_4'],
       ['HOBBIES', 'TX_1'],
       ['HOUSEHOLD', 'TX_1'],
       ['FOODS', 'TX_1'],
       ['HOBBIES', 'TX_2'],
       ['HOUSEHOLD', 'TX_2'],
       ['FOODS', 'TX_2'],
       ['HOBBIES', 'TX_3'],
       ['HOUSEHOLD', 'TX_3'],
       ['FOODS', 'TX_3'],
       ['HOBBIES', 'WI_1'],
       ['HOUSEHOLD', 'WI_1'],
       ['FOODS', 'WI_1'],
       ['HOBBIES', 'WI_2'],
       ['HOUSEHOLD', 'WI_2'],
       ['FOODS', 'WI_2'],
       ['HOBBIES', 'WI_3'],
       ['HOUSEHOLD', 'WI_3'],
       ['FOODS', 'WI_3']]

In [3]:
train_data = pd.read_csv('../data/sales_train_evaluation.csv')
calendar = pd.read_csv('../data/calendar.csv')
prices = pd.read_csv('../data/sell_prices.csv')
submission = pd.read_csv('../data/sample_submission.csv')

In [4]:
class WRMSSEEvaluator(object):

    def __init__(self, train_df: pd.DataFrame, valid_df: pd.DataFrame, 
                 calendar: pd.DataFrame, prices: pd.DataFrame):
        train_y = train_df.loc[:, train_df.columns.str.startswith('d_')]
        train_target_columns = train_y.columns.tolist()
        weight_columns = train_y.iloc[:, -28:].columns.tolist()

        train_df['all_id'] = 'all'  # for lv1 aggregation

        id_columns = train_df.loc[:, ~train_df.columns.str.startswith('d_')]\
                     .columns.tolist()
        valid_target_columns = valid_df.loc[:, valid_df.columns.str.startswith('d_')]\
                               .columns.tolist()
        
        #valid_target_columns = valid_df.columns.tolist()

        if not all([c in valid_df.columns for c in id_columns]):
            valid_df = pd.concat([train_df[id_columns], valid_df], 
                                 axis=1, sort=False)

        self.train_df = train_df
        self.valid_df = valid_df
        self.calendar = calendar
        self.prices = prices

        self.weight_columns = weight_columns
        self.id_columns = id_columns
        self.valid_target_columns = valid_target_columns

        weight_df = self.get_weight_df()

        self.group_ids = (
            'all_id',
            'state_id',
            'store_id',
            'cat_id',
            'dept_id',
            ['state_id', 'cat_id'],
            ['state_id', 'dept_id'],
            ['store_id', 'cat_id'],
            ['store_id', 'dept_id'],
            'item_id',
            ['item_id', 'state_id'],
            ['item_id', 'store_id']
        )

        for i, group_id in enumerate(tqdm(self.group_ids)):
            train_y = train_df.groupby(group_id)[train_target_columns].sum()
            scale = []
            for _, row in train_y.iterrows():
                series = row.values[np.argmax(row.values != 0):]
                scale.append(((series[1:] - series[:-1]) ** 2).mean())
            setattr(self, f'lv{i + 1}_scale', np.array(scale))
            setattr(self, f'lv{i + 1}_train_df', train_y)
            setattr(self, f'lv{i + 1}_valid_df', valid_df.groupby(group_id)\
                    [valid_target_columns].sum())

            lv_weight = weight_df.groupby(group_id)[weight_columns].sum().sum(axis=1)
            setattr(self, f'lv{i + 1}_weight', lv_weight / lv_weight.sum())

    def get_weight_df(self) -> pd.DataFrame:
        day_to_week = self.calendar.set_index('d')['wm_yr_wk'].to_dict()
        weight_df = self.train_df[['item_id', 'store_id'] + self.weight_columns]\
                    .set_index(['item_id', 'store_id'])
        weight_df = weight_df.stack().reset_index()\
                   .rename(columns={'level_2': 'd', 0: 'value'})
        weight_df['wm_yr_wk'] = weight_df['d'].map(day_to_week)

        weight_df = weight_df.merge(self.prices, how='left',
                                    on=['item_id', 'store_id', 'wm_yr_wk'])
        weight_df['value'] = weight_df['value'] * weight_df['sell_price']
        weight_df = weight_df.set_index(['item_id', 'store_id', 'd'])\
                    .unstack(level=2)['value']\
                    .loc[zip(self.train_df.item_id, self.train_df.store_id), :]\
                    .reset_index(drop=True)
        weight_df = pd.concat([self.train_df[self.id_columns],
                               weight_df], axis=1, sort=False)
        return weight_df

    def rmsse(self, valid_preds: pd.DataFrame, lv: int) -> pd.Series:
        valid_y = getattr(self, f'lv{lv}_valid_df')
        score = ((valid_y - valid_preds) ** 2).mean(axis=1)
        scale = getattr(self, f'lv{lv}_scale')
        return (score / scale).map(np.sqrt) 

    def score(self, valid_preds: Union[pd.DataFrame, 
                                       np.ndarray]) -> float:
        assert self.valid_df[self.valid_target_columns].shape \
               == valid_preds.shape

        if isinstance(valid_preds, np.ndarray):
            valid_preds = pd.DataFrame(valid_preds, 
                                       columns=self.valid_target_columns)

        valid_preds = pd.concat([self.valid_df[self.id_columns], 
                                 valid_preds], axis=1, sort=False)

        all_scores = []
        for i, group_id in enumerate(self.group_ids):

            valid_preds_grp = valid_preds.groupby(group_id)[self.valid_target_columns].sum()
            setattr(self, f'lv{i + 1}_valid_preds', valid_preds_grp)
            
            lv_scores = self.rmsse(valid_preds_grp, i + 1)
            setattr(self, f'lv{i + 1}_scores', lv_scores)
            
            weight = getattr(self, f'lv{i + 1}_weight')
            lv_scores = pd.concat([weight, lv_scores], axis=1, 
                                  sort=False).prod(axis=1)
            
            all_scores.append(lv_scores.sum())
            
        self.all_scores = all_scores

        return np.mean(all_scores)

In [5]:
def get_wrmsse_score(train_df, pred_df, split_day):
    delta = 1941 - split_day
    train_fold_df = train_df.iloc[:, : -delta]
    if delta == 28:
        valid_fold_df = train_df.iloc[:, -delta:]
    else:
        valid_fold_df = train_df.iloc[:, -delta: -delta+28]

    pred_df.rename({f'F{i}': f'd_{split_day+i}' for i in range(1,29)}, axis=1, inplace=True)

    pred_df = submission[['id']].merge(pred_df, on = 'id')

    evaluator = WRMSSEEvaluator(train_fold_df, valid_fold_df, calendar, prices)
    return evaluator.score(pred_df[[col for col in pred_df.columns if re.match('d_\d{1,4}', col)]])

In [6]:
weight_scale_df = pd.read_csv('../data/weight_scale_df.csv')
weight_scale_dict = {
    f'{item_id}_{store_id}_evaluation': weight/scale for item_id, store_id, weight, scale in weight_scale_df.values
}
weight_scale_dict

{'FOODS_1_001_CA_1_evaluation': 1.1537085781235683e-05,
 'FOODS_1_001_CA_2_evaluation': 7.73234391658877e-06,
 'FOODS_1_001_CA_3_evaluation': 4.4913467910992025e-06,
 'FOODS_1_001_CA_4_evaluation': 5.327285231787002e-06,
 'FOODS_1_001_TX_1_evaluation': 3.212129107154062e-07,
 'FOODS_1_001_TX_2_evaluation': 4.3044046130824065e-06,
 'FOODS_1_001_TX_3_evaluation': 7.321247180659534e-06,
 'FOODS_1_001_WI_1_evaluation': 8.134406104502398e-06,
 'FOODS_1_001_WI_2_evaluation': 5.393520304087257e-06,
 'FOODS_1_001_WI_3_evaluation': 6.6524804995261e-06,
 'FOODS_1_002_CA_1_evaluation': 2.8488508076891867e-05,
 'FOODS_1_002_CA_2_evaluation': 3.83300790967364e-05,
 'FOODS_1_002_CA_3_evaluation': 3.42663189572327e-05,
 'FOODS_1_002_CA_4_evaluation': 1.1251592257827941e-05,
 'FOODS_1_002_TX_1_evaluation': 7.020214310815294e-06,
 'FOODS_1_002_TX_2_evaluation': 1.0039531706153647e-05,
 'FOODS_1_002_TX_3_evaluation': 2.93023280518152e-05,
 'FOODS_1_002_WI_1_evaluation': 4.4486785591110184e-05,
 'FOODS_1

In [ ]:
lgb_params = {
    #"boosting_type": "goss",
    "n_estimators": 1000,
    "boosting_type": "gbdt",
    "objective": "tweedie",
    "tweedie_variance_power": 1.1,
    "metric": "rmse",
    "learning_rate": 0.01,
    #"num_leaves": 2 ** 5 - 1,
    #"min_data_in_leaf": 2 ** 12 - 1,
    "feature_fraction": 0.5,
    #"max_bin": 100,
    "boost_from_average": False,
    #"num_boost_round": 1400,
    "verbose": -1,
    #"num_threads": os.cpu_count(),
    "force_row_wise": True,
    "seed": 42
}

In [ ]:
%%time
all_subs = pd.DataFrame()
for cat_id, store_id in groups:
    print(f'cat_id: {cat_id}, store_id: {store_id}')
    feature_df = pd.read_csv(f'../data/cat_and_store_data/{cat_id}_and_{store_id}_features.csv')
    target_df = pd.read_csv(f'../data/cat_and_store_data/{cat_id}_and_{store_id}_target.csv')
    feature_df['sample_weight'] = feature_df.id.map(weight_scale_dict)
    select_train = train_data[(train_data['cat_id']==cat_id) & (train_data['store_id']==store_id)]
    
    price_cols = [col for col in feature_df.columns if 'price' in col]
    
    X_columns = feature_df.columns.drop(['id', 'sample_weight']).drop(price_cols)
    
    X = feature_df
    y = target_df
    
    #tsv = TimeSeriesSplit(n_splits=5, test_size=len(select_train)*90) 
    # 由于每个日期的商品个数可能略有不同，比如新上架商品，所以指定size切割的方式不合理，需要指定日期进行切割
    oof = pd.DataFrame()
    y_fits = pd.DataFrame()
    y_trues = pd.DataFrame()
    
    split_days = [1773, 1801, 1829, 1857, 1885]
    
    for i, split_day in enumerate(split_days):
        print(f'Fold {i}')
        X_train = X[X.trend<=split_day]
        y_train = y.iloc[:len(X_train)]
        X_valid = X[(X.trend>split_day) & (X.trend<=split_day+28)]
        y_valid = y.iloc[-len(X_valid):]

        model = LinearRegression()
        #model.fit(X_train[X_columns], y_train, sample_weight=X_train['sample_weight'].values)
        model.fit(X_train[X_columns], y_train)

        y_fit = pd.DataFrame(model.predict(X_train[X_columns]), index=X_train.index, columns=y.columns)
        y_pred = pd.DataFrame(model.predict(X_valid[X_columns]), index=X_valid.index, columns=y.columns)
        
        train_rmse = mean_squared_error(y_train, y_fit, squared=False)
        test_rmse = mean_squared_error(y_valid, y_pred, squared=False)
        print((f"Train RMSE: {train_rmse:.5f}\n" f"Test RMSE: {test_rmse:.5f}"))
        
        oof = pd.concat([y_pred, oof], axis=0)
        y_trues = pd.concat([y_valid, y_trues], axis=0)
        
    oof_rmse = mean_squared_error(y_trues, oof, squared=False)
    print(f"OOF RMSE: {oof_rmse:.5f}")
    
    pred_day = 1913
    pred_df = pd.concat([feature_df[-len(oof):].reset_index(), oof.reset_index()], axis=1)
    pred_df = pred_df[pred_df['trend']==pred_day]
    
    wrmsse_score = get_wrmsse_score(select_train.reset_index(), pred_df, pred_day)
    print(f"OOF WRMSSE: {wrmsse_score:.5f}")

cat_id: HOBBIES, store_id: CA_1
Fold 0
Train RMSE: 2.29995
Test RMSE: 2.25151
Fold 1
Train RMSE: 2.30106
Test RMSE: 2.21045
Fold 2
Train RMSE: 2.30075
Test RMSE: 2.20284
Fold 3
Train RMSE: 2.29952
Test RMSE: 2.19325
Fold 4
Train RMSE: 2.29737
Test RMSE: 2.17543
OOF RMSE: 2.20687


  0%|          | 0/12 [00:00<?, ?it/s]

OOF WRMSSE: 0.81962
cat_id: HOUSEHOLD, store_id: CA_1
Fold 0
Train RMSE: 1.19494
Test RMSE: 1.44074
Fold 1
Train RMSE: 1.19734
Test RMSE: 1.45105
Fold 2
Train RMSE: 1.20009
Test RMSE: 1.44671
Fold 3
Train RMSE: 1.20215
Test RMSE: 1.42030
Fold 4
Train RMSE: 1.20486
Test RMSE: 1.39921
OOF RMSE: 1.43177


  0%|          | 0/12 [00:00<?, ?it/s]

OOF WRMSSE: 0.66203
cat_id: FOODS, store_id: CA_1
Fold 0
Train RMSE: 3.33390
Test RMSE: 3.39524
Fold 1
Train RMSE: 3.32626
Test RMSE: 3.08183
Fold 2
Train RMSE: 3.31676
Test RMSE: 2.96714
Fold 3
Train RMSE: 3.30771
Test RMSE: 2.88187
Fold 4
Train RMSE: 3.30046
Test RMSE: 2.64225
OOF RMSE: 3.00412


...

OOF WRMSSE: 0.98970
cat_id: HOUSEHOLD, store_id: TX_2
Fold 0
Train RMSE: 1.83528
Test RMSE: 1.71810
Fold 1
Train RMSE: 1.83259
Test RMSE: 1.71206
Fold 2
Train RMSE: 1.82902
Test RMSE: 1.56982
Fold 3
Train RMSE: 1.82450
Test RMSE: 1.56367
Fold 4
Train RMSE: 1.81972
Test RMSE: 1.44854
OOF RMSE: 1.60594


  0%|          | 0/12 [00:00<?, ?it/s]

OOF WRMSSE: 0.79832
cat_id: FOODS, store_id: TX_2
Fold 0
Train RMSE: 3.30267
Test RMSE: 3.33915
Fold 1
Train RMSE: 3.29951
Test RMSE: 2.91127
Fold 2
Train RMSE: 3.29270
Test RMSE: 2.68433
Fold 3
Train RMSE: 3.28377
Test RMSE: 2.48982
Fold 4
Train RMSE: 3.27317
Test RMSE: 2.28018
OOF RMSE: 2.76562


  0%|          | 0/12 [00:00<?, ?it/s]

OOF WRMSSE: 0.79987
cat_id: HOBBIES, store_id: TX_3
Fold 0
Train RMSE: 1.33960
Test RMSE: 1.56595
Fold 1


In [8]:
'''
Fold 0
Train RMSE: 2.30
Test RMSE: 2.26
Fold 1
Train RMSE: 2.30
Test RMSE: 2.21
Fold 2
Train RMSE: 2.30
Test RMSE: 2.21
Fold 3
Train RMSE: 2.30
Test RMSE: 2.19
Fold 4
Train RMSE: 2.30
Test RMSE: 2.18
OOF RMSE: 2.21
100%
12/12 [00:00<00:00, 100.56it/s]
OOF WRMSSE: 0.81
CPU times: user 19min 53s, sys: 11.9 s, total: 20min 5s
Wall time: 3min 2s
'''

'\nFold 0\nTrain RMSE: 2.30\nTest RMSE: 2.26\nFold 1\nTrain RMSE: 2.30\nTest RMSE: 2.21\nFold 2\nTrain RMSE: 2.30\nTest RMSE: 2.21\nFold 3\nTrain RMSE: 2.30\nTest RMSE: 2.19\nFold 4\nTrain RMSE: 2.30\nTest RMSE: 2.18\nOOF RMSE: 2.21\n100%\n12/12 [00:00<00:00, 100.56it/s]\nOOF WRMSSE: 0.81\nCPU times: user 19min 53s, sys: 11.9 s, total: 20min 5s\nWall time: 3min 2s\n'